[Jeder kann coden](abstract/Contents.de.ipynb) / [Programmieren & TicTacToe](Programming_And_TicTacToe.de.ipynb) / [Objektorientierte Programmierung](Objectoriented_Programming.de.ipynb) / [Was sind Web APIs?](../../../WebAPIs.de.ipynb)

# Web APIs und C#

<table border="0">
  <tr>
    <td>
        <img src="WPF_n_Csharp.jpeg">
    </td>
    <td rowspan="2">
        <a href="https://miro.com/app/board/o9J_lOJi2o0=/?moveToWidget=3458764554347680798&cot=14"><img src="Radar_AccessingRestApi.jpg"></a>
    </td>
  </tr>
  <tr>
    <td>
      <a href="https://docs.gitlab.com/ee/api/members.html#list-all-members-of-a-group-or-project" target="_blank">GitLab API Doku: Mitglieder eines Projekts abrufen</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/api/system.net.http.httpclient?view=net-8.0" target="_blank">Microsoft Docs: HttpClient in C# verwenden</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/api/system.text.json.jsonserializer?view=net-8.0" target="_blank">Microsoft Docs: System.Text.Json – JSON-Daten in C# verarbeiten</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/desktop/wpf/controls/combobox-overview?view=netdesktop-8.0" target="_blank">Microsoft Docs: ComboBox in WPF</a><br>
      <a href="https://www.thecodebuzz.com/gitlab-rest-api-csharp-crud-operations/" target="_blank">Beispielartikel: GitLab REST API mit C# verwenden</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/desktop/wpf/getting-started/?view=netdesktop-8.0&WT.mc_id=dotnet-35129-website" target="_blank">WPF-Übersicht für Einsteiger</a>
    </td>
  </tr>
</table>



Hier ist eine strukturierte und kommentierte Version eines Beispiels zur Demonstration von **REST-APIs**, **HTTP-Parametern**, **GitLab-Zugriff** und **WPF-Bedienoberfläche mit C#**.

## 🎯 Ziel der Anwendung

Wir erstellen eine kleine WPF-App, in der Studierende Mitspieler aus einem GitLab-Repository per Dropdown auswählen können. Die Daten werden dabei **per REST-API** vom GitLab-Server abgerufen.

## 🧾 Übersicht der Bestandteile

1. **Modellklasse (`GitLabUser`)** zur Repräsentation eines Benutzers  
2. **Serviceklasse (`GitLabService`)** für den API-Zugriff  
3. **WPF-Oberfläche (`MainWindow.xaml`)** mit zwei ComboBoxen  
4. **Code-Behind (`MainWindow.xaml.cs`)** zur Interaktion

### 1. Modellklasse – `GitLabUser.cs`

```csharp
public class GitLabUser
{
    public int id { get; set; }
    public string name { get; set; }     // Kompletter Name, z. B. "Anna Schmidt"
    public string username { get; set; } // GitLab-Benutzername, z. B. "annas"
}
```

> 💡 Diese Klasse wird beim Parsen der JSON-Antwort der GitLab-API verwendet. C# nutzt `System.Text.Json` für das automatische Mapping.

### 2. GitLabService – `GitLabService.cs`

```csharp
public class GitLabService
{
    private readonly string _baseUrl = "https://gitlab.rz.htw-berlin.de/api/v4";

    public async Task<List<GitLabUser>> GetProjectMembersAsync(string projectId, string token)
    {
        // URL mit HTTP-Parameter "access_token"
        var url = $"{_baseUrl}/projects/{projectId}/members/all?access_token={token}";

        using var client = new HttpClient();
        var response = await client.GetAsync(url);
        response.EnsureSuccessStatusCode();

        var json = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<List<GitLabUser>>(json);
    }
}
```

> 💡 **Lehrhinweis:**  
> - Hier wird ein **HTTP-GET-Aufruf** demonstriert.  
> - `access_token` wird **als URL-Parameter** angehängt – einfach zu erklären und transparent.  
> - Zeige optional, wie man stattdessen einen **HTTP-Header** mit `Authorization: Bearer <TOKEN>` nutzen könnte.

### 3. Benutzeroberfläche – `MainWindow.xaml`

```xml
<Window x:Class="GitLabSpiel.MainWindow"
        xmlns="http://schemas.microsoft.com/winfx/2006/xaml/presentation"
        Title="Spieler Auswahl" Height="250" Width="300">
    <StackPanel Margin="10">
        <TextBlock Text="Spieler 1 wählen:" />
        <ComboBox x:Name="Player1ComboBox" DisplayMemberPath="Name" Margin="0,5"/>

        <TextBlock Text="Spieler 2 wählen:" />
        <ComboBox x:Name="Player2ComboBox" DisplayMemberPath="Name" Margin="0,5"/>

        <Button Content="Mitspieler laden" Click="Laden_Click" Margin="0,10"/>
    </StackPanel>
</Window>
```

> 💡 **Lehrhinweis:**  
> - `ComboBox` ist ein Drop-Down-Menü.  
> - `DisplayMemberPath="Name"` sorgt dafür, dass die `Name`-Eigenschaft im UI erscheint.  
> - Button triggert den API-Aufruf und lädt die Spieler.

### 4. Code-Behind – `MainWindow.xaml.cs`

```csharp
public partial class MainWindow : Window
{
    private readonly GitLabService _gitLabService = new GitLabService();

    public MainWindow()
    {
        InitializeComponent();
    }

    private async void Laden_Click(object sender, RoutedEventArgs e)
    {
        // 👉 Projekt-ID und Access-Token hier eingeben
        string projectId = "123456";      // Beispielhafte Projekt-ID
        string accessToken = "DEIN_TOKEN"; // Persönlicher GitLab Access Token

        try
        {
            var users = await _gitLabService.GetProjectMembersAsync(projectId, accessToken);
            Player1ComboBox.ItemsSource = users;
            Player2ComboBox.ItemsSource = users;
        }
        catch (Exception ex)
        {
            MessageBox.Show($"Fehler beim Laden der Benutzer:\n{ex.Message}");
        }
    }
}
```

> 💡 **Lehrhinweis:**  
> - `async/await` wird verwendet, um den API-Aufruf nicht blockierend auszuführen.  
> - Fehlerbehandlung mit `try/catch` zeigt gute Praxis.  
> - ComboBoxen erhalten direkt die Liste der Benutzer.

## 📚 Erweiterungsmöglichkeiten für Übungen

- Spieler dürfen sich nicht doppelt wählen → ComboBox 2 filtert Auswahl von ComboBox 1
- Anzeige von `Username` als Tooltip
- Avatare laden (GitLab API liefert ggf. auch Bild-URLs)
- JSON-Ergebnis im Debug-Fenster anzeigen